#### Creating a Dataframe

Let us initiate the dataframe with the first 5000 rows.

In [1]:
import pandas as pd
import numpy as np
# Store the first 5000 rows as a dataframe
ds = pd.read_csv('adult.csv')
sp_ds = ds[:5000]
df = pd.DataFrame(sp_ds)

# Task 1

>Check for missing values in the dataset and handle them \[...\] ([1.1.](#Subtask-1.1.-Checking-for-missing-values-and-handling-them))

>Check the dataset for noisy data, inconsistencies, and duplicate entries \[...\]. ([1.2.](#Subtask-1.2.-Checking-for-noisy-data,-inconsistencies-&-duplicate-entries))

## Subtask 1.1. Checking for missing values and handling them

#### Counting the missing values

There are missing values in the form of question marks ("?").

Let us [count](https://stackoverflow.com/questions/20076195/) these missing values.

In [2]:
# Stack all values and count the question marks
column_names = ["age", "workclass", "fnlwgt", "education", "educational-num", "marital-status", "occupation", "relationship", "race", "gender", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
df[[i for i in column_names]].astype('str').stack().value_counts()['?'].sum()

707

#### Counting the Null values

As seen above, there are 707 missing values in total.

Let us count the null values (if any):

In [3]:
# Check for null values
df.isnull().sum().sum()

0

#### Listing the affected columns (variables)

So there are no null values.

As for the 707 missing values found earlier, we need to know how to impute them. Let us [find](https://stackoverflow.com/questions/50923707) the columns that contain missing values.

In [4]:
# List the columns that contain a question mark value
df.columns[df.isin(['?']).any()]

Index(['workclass', 'occupation', 'native-country'], dtype='object')

#### Determining their most frequent category

The 707 missing values are distributed over three variables. All three of these are nominal variables.

For now, we will impute them using the most frequent category. Since the 707 missing values are distributed over a total of 5'000x3 = 15'000 values, this simple imputation method should not significantly affect the data.

Let us first find out the most frequent category in each variable.

In [5]:
# Count most frequent value in "workclass"
df.workclass.value_counts().head(1)

workclass
Private    3420
Name: count, dtype: int64

In [6]:
# Count most frequent value in "occupation"
df.occupation.value_counts().head(1)

occupation
Prof-specialty    644
Name: count, dtype: int64

In [7]:
# Count most frequent value in "native-country"
df['native-country'].value_counts().head(1)

native-country
United-States    4514
Name: count, dtype: int64

#### Imputing with their most frequent category

Now we know that the most frequent categories are the Private sector, the "Prof-specialty" occupation, and the country United States.

Let us replace the missing values with the respective most frequent category.

In [8]:
# Workclass : Replace question marks
df.workclass.replace('?','Private', inplace=True)
# Count most frequent value again
df.workclass.value_counts().head(1)

workclass
Private    3735
Name: count, dtype: int64

In [9]:
# Occupation : Replace question marks
df.occupation.replace('?','Prof-specialty', inplace=True)
# Count most frequent value again
df.occupation.value_counts().head(1)

occupation
Prof-specialty    959
Name: count, dtype: int64

In [10]:
# Native-country : Replace question marks
df['native-country'].replace('?','United-States', inplace=True)
# Count most frequent value again
df['native-country'].value_counts().head(1)

native-country
United-States    4591
Name: count, dtype: int64

#### Checking that the missing values are handled

As expected, the most frequent categories became even more frequent. Let us check for missing values again.

In [11]:
# List the columns that contain a question mark value
df.columns[df.isin(['?']).any()]

Index([], dtype='object')

## Subtask 1.2. Checking for noisy data, inconsistencies & duplicate entries

### Subsubtask 1.2.1 Checking for noisy data

Some definitions for noisy data include: Data that is corrupted/distorted & Data that has a low signal-to-noise ratio.

While the latter can be [measured using a formula](https://en.wikipedia.org/wiki/Signal-to-noise_ratio), checking for corrupted and distorted data will need manual examination.

#### Finding corrupted data

[There are 8 nominal variables in our data.](http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html) Corrupted values are likely to hide in their rarest categories.

Let us search for corrupted data, by listing rare categories in non-number variables, treating "native-country" separately since it has especially many categories.

In [12]:
# Stack values in said non-number variables and show the rarest categories
non_num_column_names = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender"]
df[[i for i in non_num_column_names]].astype('str').stack().value_counts().tail()

1st-4th              24
Preschool            13
Married-AF-spouse     3
Without-pay           2
Armed-Forces          1
Name: count, dtype: int64

In [13]:
# Stack values in "native-country" and show the rarest categories
df[['native-country']].astype('str').stack().value_counts().tail()

Laos                          1
Hong                          1
Trinadad&Tobago               1
Outlying-US(Guam-USVI-etc)    1
France                        1
Name: count, dtype: int64

Examining these values reveals no corrupted information except for typing errors in the country names of Hong Kong and Trinidad&Tobago. However, the typing errors do not affect the usefulness of the data, as each country is represented by only one category.

#### Finding distorted data

[Our data has 6 continous variables.](http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html) They are "age", "fnlwgt" (finalweight), "educational-num", "capital-gain", "capital-loss", and "hours-per-week". For the first five, there exists a distribution plot in our first presentation. As for "hours-per-week", a plot has been generated below.

Considering all of these plots, there is only one obvious distortion: Age 90 is more represented than the other ages in that range. We assume that this is an intended grouping of ages 90+ that has been made in the original data. Therefore it does not negatively affect the quality of the dataset

In [ ]:
# Value distribution plot for "hours-per-week" (requires matplotlib)
df['hours-per-week'].plot.hist()

#### Measuring noise in numeric variables

We will measure the signal-to-noise ratio (SNR) of the 6 numeric variables to determine if any has a high noise.

According to [this Wikipedia article](https://en.wikipedia.org/wiki/Signal-to-noise_ratio), the SNR can be calculated as (μ^2/σ^2).

Let us find μ (the means) and σ (the standard deviations) to apply the formula.

In [14]:
# Get mean and STD of the the numerical variables
numericInfo = df.describe().iloc[1:3]
numericInfo

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
mean,38.504800,187269.876000,10.038400,1107.090200,89.561400,40.20600
std,13.779781,102781.411899,2.595433,7708.019951,408.651437,12.45805


Now, let us compute the SNR.

In [15]:
# Calculate SNRs and store them in variables
resultage = ( (numericInfo['age'][0])**2 ) / ( (numericInfo['age'][1])**2 )
resultweight = ( (numericInfo['fnlwgt'][0])**2 ) / ( (numericInfo['fnlwgt'][1])**2 )
resultedu = ( (numericInfo['educational-num'][0])**2 ) / ( (numericInfo['educational-num'][1])**2 )
resultcapg = ( (numericInfo['capital-gain'][0])**2 ) / ( (numericInfo['capital-gain'][1])**2 )
resultcapl = ( (numericInfo['capital-loss'][0])**2 ) / ( (numericInfo['capital-loss'][1])**2 )
resulthpw = ( (numericInfo['hours-per-week'][0])**2 ) / ( (numericInfo['hours-per-week'][1])**2 )
# Output the variables
def SNRs():
    return {"Age": resultage, "Final Weight": resultweight, "Education-Num": resultedu, "Capital Gain": resultcapg, "Capital Loss": resultcapl, "Hours per Week": resulthpw}
SNRs()

{'Age': 7.8080958880742415,
 'Final Weight': 3.319759996517732,
 'Education-Num': 14.959233262883458,
 'Capital Gain': 0.02062910339656595,
 'Capital Loss': 0.04803255474767841,
 'Hours per Week': 10.415535705359764}

As we can see, the signal is significantly stronger than the noise for most variables. However, the two variables "capital-gain" and "capital-loss" have a very weak SNR.

The high noise in the capital related data might make its analysis difficult for machine learning (ML) models. We will approach this problem with either feature scaling or feature engineering in Task 3.

### Subsubtask 1.2.2 Checking for inconsistencies

Let us make sure Capital Gains VS Losses are mutually exclusive.

In [16]:
# Output entries with non-zero capital gains AND losses, if any
def capitalInconsistencies():
    inconsistent_data = df[(df['capital-gain'] > 0) & (df['capital-loss'] > 0)]
    if not inconsistent_data.empty:
        return inconsistent_data
    else:
        return "Empty Dataset"
capitalInconsistencies()

'Empty Dataset'

We note that there is no entry with capital gains AND losses.

### Subsubtask 1.2.3 Checking for duplicates

In [17]:
# Output entries that are identical, if any
def duplicatesInDf():
    duplicates = df[df.duplicated(keep=False)]
    if not duplicates.empty:
        return duplicates
    else:
        return "Empty Dataset"
duplicatesInDf()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
488,24,Private,194630,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,35,United-States,<=50K
864,24,Private,194630,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,35,United-States,<=50K


Seeing as there is one duplicate, let us drop it.

In [18]:
# Remove duplicates
df.drop_duplicates(inplace=True)
# Search for duplicates
duplicatesInDf()

'Empty Dataset'

In [19]:
# Make sure that there are still no missing values somehow
assert df.isnull().sum().sum() == 0
True

True

# Task 2

>[Convert categorical features to numerical values using encoding techniques]

#### Encoding education

As already mentioned in [1.2.1](#Subsubtask-1.2.1-Checking-for-noisy-data), there are 8 nominal variables. Education is already ordinally encoded, so let us drop the non-numerical variable.

In [20]:
# Remove variable "education"

#### Encoding workclass, marital-status, occupation, relationship, race, gender

Since the other nominal variables are not ordinal, we do not want to use e.g. label encoding, as that could introduce unintended biases. Instead, we will use One-Hot encoding, for which we will need to map a category to an integer.

In [21]:
# One-hot Encoding for 6 of the nominal features

#### Encoding native-country

The only exception will be "native-country" which has many categories. Therefore we will use target encoding.

In [22]:
# Target encode "native-country"